In [3]:
import pandas as pd
import re 
import os, sys, email
#import gensim
#from gensim.models import Doc2Vec
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from string import punctuation
import timeit
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import pylab as pl

In [67]:
start = timeit.default_timer()
emails_df = pd.read_csv('emails.csv')
enron_df = pd.read_csv('enron_cleaned2.csv')

In [5]:
## This code in this and next cell is taken from the from Explore Enron notebook by Zichen Wang
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [6]:
messages = list(map(email.message_from_string,emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

emails_df['Subjcontent'] = emails_df['Subject'] + " " + emails_df['content']
emails_df.head(5)

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,Subjcontent
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,Re: Traveling to have a business meeting takes...
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p,Re: test test successful. way to go!!!
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p,"Randy,\n\n Can you send me a schedule of the ..."
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p,Re: Hello Let's shoot for Tuesday at 11:45.


### String Matching - Always take the First word(family name) from excel name as other names are very common.
### You will get some non our person emails like 2-3 instances for each name will be someone else but ignore that does not matter much

In [ ]:
emails_df['X-From'][emails_df['X-From'].str.contains("BELDEN".title())==True].unique()

In [7]:
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("GARLAND".title())==True] = 'GARLAND C KEVIN'

In [8]:
#Separate the data from original set into name set. take only X-From and Content 
Garland_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("GARLAND")==True]
Garland_test.head()

,X-From,content
38622,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
39036,GARLAND C KEVIN,"Rick,\n\nOur group has completed our new inves..."
39050,GARLAND C KEVIN,We will wait to you get back. I had heard of ...
188260,GARLAND C KEVIN,"Notice Number: 01-440\nDecember 21, 2001\n\nT..."
204696,GARLAND C KEVIN,I will ask Rebekah to try to set it up for Mon...


In [9]:
#Join all the contents into one
Garland_test = Garland_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()

In [10]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("GATHMANN".title())==True] = 'GATHMANN WILLIAM D'
#Separate the data from original set into name set. take only X-From and Content 
Gathmann_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("GATHMANN")==True]
Gathmann_test.head()

,X-From,content
227468,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
232258,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
238569,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
248094,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
250451,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."


In [11]:
#Join all the contents into one
Gathmann_test = Gathmann_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
Gathmann_test.head()

,X-From,content
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."


### Now Combine two data frames into email_data

In [12]:
email_data = pd.concat([Garland_test, Gathmann_test], axis=0)

In [13]:
email_data.head()

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."


In [15]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("GIBBS".title())==True] = 'GIBBS DANA R'
#Separate the data from original set into name set. take only X-From and Content 
GIBBS_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("GIBBS")==True]
print(GIBBS_test.head())
GIBBS_test = GIBBS_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
GIBBS_test.head()

             X-From                                            content
47105  GIBBS DANA R           \n@ Receiver drop Pathon  pickup Pollard
47193  GIBBS DANA R        \nI take Jeff Graham n  drop Koren Robinson
47208  GIBBS DANA R  \nDrop Kina (that useless  piece o Turkey Turd...
47511  GIBBS DANA R        \nI take Jeff Graham n  drop Koren Robinson
96791  GIBBS DANA R  Ladies and Gentlemen:\n\nA copy of the NERC Ne...


,X-From,content
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...


In [16]:
email_data = pd.concat([email_data, GIBBS_test],axis=0)

In [17]:
email_data.head()

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...


In [43]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("GILLIS".title())==True] = 'GILLIS JOHN'
#Separate the data from original set into name set. take only X-From and Content 
GILLIS_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("GILLIS")==True]
print(GILLIS_test.head())
GILLIS_test = GILLIS_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
GILLIS_test.head()

             X-From                                            content
230641  GILLIS JOHN  >Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n...
230685  GILLIS JOHN  Steve,\n\n Here are my first-cut comments on t...
232728  GILLIS JOHN  Steve,\n\n Here are my first-cut comments on t...
232732  GILLIS JOHN  >Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n...
237476  GILLIS JOHN  Steve,\n\n Here are my first-cut comments on t...


,X-From,content
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."


In [21]:
email_data = pd.concat([email_data, GILLIS_test],axis=0)
email_data.head()

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."


In [29]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("GRAMM".title())==True] = 'GRAMM WENDY L'

#Separate the data from original set into name set. take only X-From and Content 
GRAMM_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("GRAMM WENDY L")==True]
GRAMM_test = GRAMM_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
GRAMM_test.head()

,X-From,content
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."


In [31]:
email_data = pd.concat([email_data, GRAMM_test],axis=0)
email_data.head(6)

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."


In [33]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("GRAY".title())==True] = 'GRAY RODNEY'
print(GILLIS_test.head())


#Separate the data from original set into name set. take only X-From and Content 
GRAY_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("GRAY RODNEY")==True]
GRAY_test = GRAY_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
GRAY_test.head()

        X-From                                            content
0  GILLIS JOHN  >Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n...


,X-From,content
0,GRAY RODNEY,"\n\n Ladies,\n\nLooks we have been assigned Co..."


In [34]:
email_data = pd.concat([email_data, GRAY_test],axis=0)
email_data.head(6)

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."


In [39]:
type(email_data)

pandas.core.frame.DataFrame

In [41]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("HAEDICKE".title())==True] = 'HAEDICKE MARK E'

#Separate the data from original set into name set. take only X-From and Content 
HAEDICKE_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("HAEDICKE MARK E")==True]
print(HAEDICKE_test.head())
HAEDICKE_test = HAEDICKE_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
HAEDICKE_test.head()

                X-From                                            content
5011   HAEDICKE MARK E  \tIn order to bring better legal coordination ...
29422  HAEDICKE MARK E  \nOnce again this year, I am leading the BEAR ...
43897  HAEDICKE MARK E  \nOnce again this year, I am leading the BEAR ...
44292  HAEDICKE MARK E  \tIn order to bring better legal coordination ...
47943  HAEDICKE MARK E  How would we do this efficiently in EWS?  Mark...


,X-From,content
0,HAEDICKE MARK E,\tIn order to bring better legal coordination ...


In [42]:
email_data = pd.concat([email_data, HAEDICKE_test],axis=0)
email_data.head(6)

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."


In [44]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("CAUSEY".title())==True] = 'CAUSEY RICHARD A'

#Separate the data from original set into name set. take only X-From and Content 
CAUSEY_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("CAUSEY RICHARD A")==True]
print(CAUSEY_test.head())
CAUSEY_test = CAUSEY_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
CAUSEY_test.head()

                 X-From                                            content
22594  CAUSEY RICHARD A  I just looked over your draft of the fundament...
22696  CAUSEY RICHARD A  Plenty to do.  They should definitely be on SA...
23367  CAUSEY RICHARD A  Good, valid points.  I think that I agree that...
23375  CAUSEY RICHARD A  I agree fully with the last sentence.  E mail ...
23753  CAUSEY RICHARD A  As many of you know, The University of Texas h...


,X-From,content
0,CAUSEY RICHARD A,I just looked over your draft of the fundament...


In [46]:
email_data = pd.concat([email_data, CAUSEY_test],axis=0)
email_data.head(9)

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAY RODNEY,"\n\n Ladies,\n\nLooks we have been assigned Co..."
0,HAEDICKE MARK E,\tIn order to bring better legal coordination ...


In [47]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("COLWELL".title())==True] = 'COLWELL WESLEY'

#Separate the data from original set into name set. take only X-From and Content 
COLWELL_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("COLWELL WESLEY")==True]
print(COLWELL_test.head())
COLWELL_test = COLWELL_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
COLWELL_test.head()

               X-From                                            content
22804  COLWELL WESLEY  Sorry I am so late in replying!  It would seem...
24859  COLWELL WESLEY  Sorry I am so late in replying!  It would seem...
25262  COLWELL WESLEY  JUST A FRIENDLY REMINDER!\n\nWholesale Service...
29032  COLWELL WESLEY  I believe you will still have to allocate.  We...
29177  COLWELL WESLEY  I still have some concern over the discussions...


,X-From,content
0,COLWELL WESLEY,Sorry I am so late in replying! It would seem...


In [49]:
email_data = pd.concat([email_data, COLWELL_test],axis=0)
email_data.head(12)

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAY RODNEY,"\n\n Ladies,\n\nLooks we have been assigned Co..."
0,HAEDICKE MARK E,\tIn order to bring better legal coordination ...
0,CAUSEY RICHARD A,I just looked over your draft of the fundament...


In [50]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("DELANEY".title())==True] = 'DELAINEY DAVID W'

#Separate the data from original set into name set. take only X-From and Content 
DELANEY_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("DELAINEY DAVID W")==True]
print(DELANEY_test.head())
DELANEY_test = DELANEY_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
DELANEY_test.head()

                X-From                                            content
8809  DELAINEY DAVID W  Below is a summary of my SWPTF, CAISO and SW I...
8833  DELAINEY DAVID W  tom, attached is the agenda and the documents ...
8991  DELAINEY DAVID W  Below is a summary of my SWPTF, CAISO and SW I...
9005  DELAINEY DAVID W  tom, attached is the agenda and the documents ...
9192  DELAINEY DAVID W  tom, attached is the agenda and the documents ...


,X-From,content
0,DELAINEY DAVID W,"Below is a summary of my SWPTF, CAISO and SW I..."


In [51]:
email_data = pd.concat([email_data, DELANEY_test],axis=0)
email_data.head(13)

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAY RODNEY,"\n\n Ladies,\n\nLooks we have been assigned Co..."
0,HAEDICKE MARK E,\tIn order to bring better legal coordination ...
0,CAUSEY RICHARD A,I just looked over your draft of the fundament...


In [52]:
email_data.drop_duplicates(subset=None, keep='first', inplace=True)

In [53]:
email_data

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAY RODNEY,"\n\n Ladies,\n\nLooks we have been assigned Co..."
0,HAEDICKE MARK E,\tIn order to bring better legal coordination ...
0,CAUSEY RICHARD A,I just looked over your draft of the fundament...
0,COLWELL WESLEY,Sorry I am so late in replying! It would seem...
0,DELAINEY DAVID W,"Below is a summary of my SWPTF, CAISO and SW I..."


In [54]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("FASTOW".title())==True] = 'FASTOW ANDREW S'

#Separate the data from original set into name set. take only X-From and Content 
FASTOW_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("FASTOW ANDREW S")==True]
print(FASTOW_test.head())
FASTOW_test = FASTOW_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
FASTOW_test.head()

                 X-From                                            content
257950  FASTOW ANDREW S  Louise:\n\nDo you have any interest in attendi...
257953  FASTOW ANDREW S  Louise:\n\nAs an additional follow-up, as part...
257954  FASTOW ANDREW S  Louise:\n\nI am NOT taking the P/L swings or f...
258928  FASTOW ANDREW S  John:\n\nThanks for the e-mail.  I wasn't tryi...
271885  FASTOW ANDREW S  \n\n -----Original Message-----\nFrom: \tStabl...


,X-From,content
0,FASTOW ANDREW S,Louise:\n\nDo you have any interest in attendi...


In [55]:
email_data = pd.concat([email_data, FASTOW_test],axis=0)

In [56]:
email_data

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAY RODNEY,"\n\n Ladies,\n\nLooks we have been assigned Co..."
0,HAEDICKE MARK E,\tIn order to bring better legal coordination ...
0,CAUSEY RICHARD A,I just looked over your draft of the fundament...
0,COLWELL WESLEY,Sorry I am so late in replying! It would seem...
0,DELAINEY DAVID W,"Below is a summary of my SWPTF, CAISO and SW I..."


In [57]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("GLISAN".title())==True] = 'GLISAN JR BEN F'

#Separate the data from original set into name set. take only X-From and Content 
GLISAN_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("GLISAN JR BEN F")==True]
print(GLISAN_test.head())
GLISAN_test = GLISAN_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
GLISAN_test.head()

                 X-From                                            content
38372   GLISAN JR BEN F  George & Gordon\n\nPlease work with RAC (Dave ...
161226  GLISAN JR BEN F  Attached please find my model for GPG leverage...
161306  GLISAN JR BEN F  Attached please find my model for GPG leverage...
161994  GLISAN JR BEN F  Attached please find my model for GPG leverage...
256545  GLISAN JR BEN F  Thanks for the note.  The concern was not over...


,X-From,content
0,GLISAN JR BEN F,George & Gordon\n\nPlease work with RAC (Dave ...


In [59]:
email_data = pd.concat([email_data, GLISAN_test],axis=0)

In [60]:
email_data


,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAY RODNEY,"\n\n Ladies,\n\nLooks we have been assigned Co..."
0,HAEDICKE MARK E,\tIn order to bring better legal coordination ...
0,CAUSEY RICHARD A,I just looked over your draft of the fundament...
0,COLWELL WESLEY,Sorry I am so late in replying! It would seem...
0,DELAINEY DAVID W,"Below is a summary of my SWPTF, CAISO and SW I..."


In [61]:
email_data.drop_duplicates(subset=None, keep='first', inplace=True)
email_data

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAY RODNEY,"\n\n Ladies,\n\nLooks we have been assigned Co..."
0,HAEDICKE MARK E,\tIn order to bring better legal coordination ...
0,CAUSEY RICHARD A,I just looked over your draft of the fundament...
0,COLWELL WESLEY,Sorry I am so late in replying! It would seem...
0,DELAINEY DAVID W,"Below is a summary of my SWPTF, CAISO and SW I..."


In [62]:
#Repeat for next name
#Assign proper name from excel
emails_df['X-From'][emails_df['X-From'].str.contains("HANNON".title())==True] = 'HANNON KEVIN P'

#Separate the data from original set into name set. take only X-From and Content 
HANNON_test = emails_df[['X-From','content']][emails_df['X-From'].str.contains("HANNON KEVIN P")==True]
print(HANNON_test.head())
HANNON_test = HANNON_test.groupby('X-From')['content'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
HANNON_test.head()

                X-From                                            content
32862   HANNON KEVIN P  \nFrom Kevin Hannon, Jim Hughes and John Lavor...
39043   HANNON KEVIN P  \nFrom Kevin Hannon, Jim Hughes and John Lavor...
44855   HANNON KEVIN P  \nFrom Kevin Hannon, Jim Hughes and John Lavor...
92851   HANNON KEVIN P  To:  Enron Corporate Policy Committee:\n\nPlea...
172939  HANNON KEVIN P  To:  Enron Corporate Policy Committee:\n\nPlea...


,X-From,content
0,HANNON KEVIN P,"\nFrom Kevin Hannon, Jim Hughes and John Lavor..."


In [63]:
email_data = pd.concat([email_data, HANNON_test],axis=0)

In [64]:
email_data

,X-From,content
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...
0,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o..."
0,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...
0,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n..."
0,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'..."
0,GRAY RODNEY,"\n\n Ladies,\n\nLooks we have been assigned Co..."
0,HAEDICKE MARK E,\tIn order to bring better legal coordination ...
0,CAUSEY RICHARD A,I just looked over your draft of the fundament...
0,COLWELL WESLEY,Sorry I am so late in replying! It would seem...
0,DELAINEY DAVID W,"Below is a summary of my SWPTF, CAISO and SW I..."


In [74]:
features_df = email_data.merge(right = enron_df, left_on="X-From", right_on="name")

In [76]:
features_df.head()

,X-From,content,name,poi,salary,bonus,long_term_incentive,deferred_income,deferral_payments,loan_advances,...,total_payments,exercised_stock_options,restricted_stock,restricted_stock_deferred,total_stock_value,to_messages,from_messages,from_poi_to_this_person,from_this_person_to_poi,shared_receipt_with_poi
0,GARLAND C KEVIN,\n\nAnnouncement\n\nKevin Garland has joined t...,GARLAND C KEVIN,False,231946,850000.0,375304.0,0.0,0,0.0,...,1566469,636246,259907,0,896153,209.000000,44.000000,10.0,27.000000,178.000000
1,GATHMANN WILLIAM D,"Richard,\n\nSince the only approvals we need o...",GATHMANN WILLIAM D,False,0,0.0,0.0,0.0,0,0.0,...,0,1753766,264013,-72419,1945360,2007.111111,668.763889,58.5,36.277778,1058.527778
2,GIBBS DANA R,\n@ Receiver drop Pathon pickup Pollard \nI t...,GIBBS DANA R,False,0,0.0,461912.0,0.0,504610,0.0,...,966522,2218275,0,0,2218275,169.000000,12.000000,0.0,0.000000,23.000000
3,GILLIS JOHN,">Date: Mon, 22 May 2000 15:43:36 -0500 (CDT)\n...",GILLIS JOHN,False,0,0.0,0.0,0.0,0,0.0,...,0,9803,75838,0,85641,2007.111111,668.763889,58.5,36.277778,1058.527778
4,GRAMM WENDY L,"Hi Steve,\nFolks love Newsome, and I think he'...",GRAMM WENDY L,False,0,0.0,0.0,0.0,0,0.0,...,119292,0,0,0,0,2007.111111,668.763889,58.5,36.277778,1058.527778
